In [1]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time

In [2]:
start_time = time.time()
# 时间处理
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"

In [3]:
# 导入数据
# Target log path
logs_path = './tmp/rnn_words'
writer = tf.summary.FileWriter(logs_path)

# Text file containing words for training
training_file = 'belling_the_cat.txt'

def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [content[i].split() for i in range(len(content))]
    content = np.array(content)
    content = np.reshape(content, [-1, ])
    return content

training_data = read_data(training_file)
print("Loaded training data...")

Loaded training data...


In [4]:
# 2.建立字典和逆序字典的函数
# 预测值也是一个唯一的整数值与逆序字典中预测符号的索引相对应。
count = collections.Counter(training_data).most_common()
#print(collections.Counter(training_data).most_common())
dictionary = dict()
for word, _ in count:
    dictionary[word] = len(dictionary)
reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
print(dictionary)
print('*'*100)
print(reverse_dictionary)

{',': 0, 'the': 1, '.': 2, 'and': 3, 'to': 4, 'said': 5, 'a': 6, 'could': 7, 'that': 8, 'cat': 9, 'some': 10, 'this': 11, 'mouse': 12, 'he': 13, 'in': 14, 'we': 15, 'is': 16, 'mice': 17, 'had': 18, 'general': 19, 'enemy': 20, 'but': 21, 'at': 22, 'got': 23, 'up': 24, 'proposal': 25, 'which': 26, 'all': 27, 'of': 28, 'her': 29, 'easily': 30, 'propose': 31, 'bell': 32, 'by': 33, 'she': 34, 'was': 35, 'old': 36, 'long': 37, 'ago': 38, 'council': 39, 'consider': 40, 'what': 41, 'measures': 42, 'they': 43, 'take': 44, 'outwit': 45, 'their': 46, 'common': 47, 'last': 48, 'young': 49, 'make': 50, 'thought': 51, 'would': 52, 'meet': 53, 'case': 54, 'you': 55, 'will': 56, 'agree': 57, 'our': 58, 'chief': 59, 'danger': 60, 'consists': 61, 'sly': 62, 'treacherous': 63, 'manner': 64, 'approaches': 65, 'us': 66, 'now': 67, 'if': 68, 'receive': 69, 'signal': 70, 'approach': 71, 'escape': 72, 'from': 73, 'i': 74, 'venture': 75, 'therefore': 76, 'small': 77, 'be': 78, 'procured': 79, 'attached': 80, '

In [5]:
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

In [6]:
# 3.常量和训练参数
dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)

# Parameters
learning_rate = 0.001
training_iters = 50000
display_step = 1000
n_input = 3

# number of units in RNN cell
n_hidden = 512

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

In [7]:
# 4.LSTM 单元的网络模型
def RNN(x, weights, biases):

    # reshape to [1, n_input]
    # static_rnn
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)
    # static_rnn => [batch_size,time_step_size,1]

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [8]:
# tensorflow 执行
# Launch the graph
#  模型训练
pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        # 2D 
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input,1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
            #save model
            saver = tf.train.Saver()
            saver.save(session, "./model/lstem_text.model", global_step=step)
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("Point your web browser to: http://localhost:6006/")
    while True:
        prompt = "%s words: " % n_input
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Iter= 1000, Average Loss= 4.239081, Average Accuracy= 6.20%
['is', 'all', 'very'] - [well] vs [with]
Iter= 2000, Average Loss= 2.747463, Average Accuracy= 25.80%
['old', 'mouse', 'got'] - [up] vs [up]
Iter= 3000, Average Loss= 2.707682, Average Accuracy= 30.10%
['.', 'this', 'proposal'] - [met] vs [at]
Iter= 4000, Average Loss= 2.119124, Average Accuracy= 47.70%
['she', 'was', 'about'] - [,] vs [,]
Iter= 5000, Average Loss= 1.867641, Average Accuracy= 53.70%
['by', 'this', 'means'] - [we] vs [we]
Iter= 6000, Average Loss= 1.452327, Average Accuracy= 60.30%
[',', 'therefore', ','] - [to] vs [to]
Iter= 7000, Average Loss= 1.352631, Average Accuracy= 65.60%
['easily', 'escape', 'from'] - [her] vs [she]
Iter= 8000, Average Loss= 1.216106, Average Accuracy= 67.90

KeyboardInterrupt: 